In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext


# ALGORITHM SECTION

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right


class ServiceCenterSolver:
    def __init__(self):
        self.service_centers = 0
        self.covered = set()
        self.center_nodes = []
    
    def min_service_centers(self, root):
        self.service_centers = 0
        self.covered = {None}  # null nodes are covered by default
        self.center_nodes = []
        
        self._dfs(root, None)
        
        return self.service_centers
    
    def _dfs(self, node, parent):
        if node is None:
            return
        
        # Process left and right subtrees first
        self._dfs(node.left, node)
        self._dfs(node.right, node)
        
        # If parent is None (root) and node not covered, OR
        # If any child is not covered, place service center here
        if (parent is None and node not in self.covered) or \
           (node.left not in self.covered) or \
           (node.right not in self.covered):
            
            self.service_centers += 1
            self.center_nodes.append(node.val)
            
            # This service center covers:
            # - Current node
            # - Parent node
            # - Both children
            self.covered.add(node)
            self.covered.add(parent)
            self.covered.add(node.left)
            self.covered.add(node.right)
    
    def get_solution_details(self, root):
        self.min_service_centers(root)
        
        details = {
            'total_centers': self.service_centers,
            'center_locations': self.center_nodes,
            'explanation': self._generate_explanation()
        }
        
        return details
    
    def _generate_explanation(self):
        explanation = []
        explanation.append("Algorithm: Greedy DFS (Post-order Traversal)")
        explanation.append("\nStrategy:")
        explanation.append("1. Process tree bottom-up (leaves to root)")
        explanation.append("2. If a leaf is uncovered, place center at its parent")
        explanation.append("3. Each center covers: the node, its parent, and its children")
        explanation.append(f"\nResult: {self.service_centers} service center(s) needed")
        explanation.append(f"Locations (node values): {self.center_nodes}")
        
        return "\n".join(explanation)


class TreeBuilder:
    @staticmethod
    def from_list(arr):
        if not arr or arr[0] is None:
            return None
        
        root = TreeNode(arr[0])
        queue = [root]
        i = 1
        
        while queue and i < len(arr):
            node = queue.pop(0)
            
            # Left child
            if i < len(arr) and arr[i] is not None:
                node.left = TreeNode(arr[i])
                queue.append(node.left)
            i += 1
            
            # Right child
            if i < len(arr) and arr[i] is not None:
                node.right = TreeNode(arr[i])
                queue.append(node.right)
            i += 1
        
        return root
    
    @staticmethod
    def parse_input(input_str):
        # Remove brackets and whitespace
        input_str = input_str.strip().strip('[]')
        
        # Split by comma
        parts = input_str.split(',')
        
        # Convert to list with None for null
        arr = []
        for part in parts:
            part = part.strip().lower()
            if part == 'null' or part == 'none':
                arr.append(None)
            else:
                try:
                    arr.append(int(part))
                except ValueError:
                    raise ValueError(f"Invalid value: {part}")
        
        return TreeBuilder.from_list(arr)
    
    @staticmethod
    def tree_to_string(root):
        """Convert tree to string representation for display"""
        if not root:
            return "[]"
        
        result = []
        queue = [root]
        
        while queue:
            node = queue.pop(0)
            if node:
                result.append(str(node.val))
                queue.append(node.left)
                queue.append(node.right)
            else:
                result.append("null")
        
        # Remove trailing nulls
        while result and result[-1] == "null":
            result.pop()
        
        return "[" + ", ".join(result) + "]"


# GUI SECTION 

class ServiceCenterGUI:
    
    def __init__(self, root):
        self.root = root
        self.root.title("Service Center Binary Tree")
        self.root.geometry("1000x750")
        self.root.configure(bg="#0f0f23")
        
        self.tree_root = None
        self.canvas_nodes = {}
        
        self.setup_ui()
    
    def setup_ui(self):
        
        
        # Title Frame
        title_frame = tk.Frame(self.root, bg="#0f0f23")
        title_frame.pack(pady=15)
        
        title_label = tk.Label(
            title_frame,
            text="Service Center Placement",
            font=("Arial", 26, "bold"),
            bg="#0f0f23",
            fg="#00ff00"
        )
        title_label.pack()
        
        subtitle_label = tk.Label(
            title_frame,
            text="Minimum Service Centers in Binary Tree",
            font=("Arial", 11),
            bg="#0f0f23",
            fg="#888888"
        )
        subtitle_label.pack()
        
        # Input Frame
        input_frame = tk.Frame(self.root, bg="#1a1a2e", relief=tk.RAISED, bd=2)
        input_frame.pack(pady=10, padx=20, fill=tk.X)
        
        tk.Label(
            input_frame,
            text="Tree Input (level-order with 'null' for empty nodes):",
            font=("Arial", 11, "bold"),
            bg="#1a1a2e",
            fg="white"
        ).pack(pady=8)
        
        self.input_entry = tk.Entry(
            input_frame,
            font=("Courier", 12),
            width=50,
            bg="#16213e",
            fg="#00ff00",
            insertbackground="white"
        )
        self.input_entry.insert(0, "0, 0, null, 0, null, 0, null, null, 0")
        self.input_entry.pack(pady=5)
        
        help_label = tk.Label(
            input_frame,
            text="Example: 0,0,null,0,null,0,null,null,0  or  [1,2,3,null,4]",
            font=("Arial", 9),
            bg="#1a1a2e",
            fg="#666666"
        )
        help_label.pack(pady=2)
        
        # Buttons
        button_frame = tk.Frame(input_frame, bg="#1a1a2e")
        button_frame.pack(pady=10)
        
        self.solve_btn = tk.Button(
            button_frame,
            text="Solve",
            font=("Arial", 11, "bold"),
            bg="#00ff88",
            fg="black",
            width=15,
            command=self.solve_problem,
            cursor="hand2"
        )
        self.solve_btn.grid(row=0, column=0, padx=5)
        
        self.visualize_btn = tk.Button(
            button_frame,
            text="Build Tree",
            font=("Arial", 11, "bold"),
            bg="#00aaff",
            fg="black",
            width=15,
            command=self.build_and_visualize,
            cursor="hand2"
        )
        self.visualize_btn.grid(row=0, column=1, padx=5)
        
        self.clear_btn = tk.Button(
            button_frame,
            text="Clear",
            font=("Arial", 11, "bold"),
            bg="#ff4757",
            fg="white",
            width=15,
            command=self.clear_all,
            cursor="hand2"
        )
        self.clear_btn.grid(row=0, column=2, padx=5)
        
        # Result Frame
        result_frame = tk.Frame(self.root, bg="#1a1a2e", relief=tk.RAISED, bd=2)
        result_frame.pack(pady=10, padx=20, fill=tk.X)
        
        self.result_label = tk.Label(
            result_frame,
            text="Minimum Service Centers: ---",
            font=("Arial", 18, "bold"),
            bg="#1a1a2e",
            fg="#ffd700"
        )
        self.result_label.pack(pady=12)
        
        # Main container for tree and explanation
        main_container = tk.Frame(self.root, bg="#0f0f23")
        main_container.pack(pady=5, padx=20, fill=tk.BOTH, expand=True)
        
        # Tree Visualization Frame (Left side)
        tree_frame = tk.Frame(main_container, bg="#1a1a2e", relief=tk.RAISED, bd=2)
        tree_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 5))
        
        tk.Label(
            tree_frame,
            text="Tree Visualization:",
            font=("Arial", 12, "bold"),
            bg="#1a1a2e",
            fg="white"
        ).pack(pady=5)
        
        # Canvas for tree drawing
        self.canvas = tk.Canvas(
            tree_frame,
            bg="#16213e",
            width=450,
            height=400
        )
        self.canvas.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)
        
        # Explanation Frame (Right side)
        explanation_frame = tk.Frame(main_container, bg="#1a1a2e", relief=tk.RAISED, bd=2)
        explanation_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=(5, 0))
        
        tk.Label(
            explanation_frame,
            text="Algorithm Explanation:",
            font=("Arial", 12, "bold"),
            bg="#1a1a2e",
            fg="white"
        ).pack(pady=5)
        
        self.explanation_text = scrolledtext.ScrolledText(
            explanation_frame,
            font=("Courier", 10),
            bg="#16213e",
            fg="#00ff00",
            height=15,
            wrap=tk.WORD
        )
        self.explanation_text.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)
        
        # Legend
        legend_frame = tk.Frame(self.root, bg="#1a1a2e", relief=tk.RAISED, bd=2)
        legend_frame.pack(pady=5, padx=20, fill=tk.X)
        
        tk.Label(
            legend_frame,
            text="Legend: Service Center  | Regular Node",
            font=("Arial", 10),
            bg="#1a1a2e",
            fg="white"
        ).pack(pady=5)
    
    def build_and_visualize(self):
        """Build tree from input and visualize it"""
        try:
            input_text = self.input_entry.get()
            self.tree_root = TreeBuilder.parse_input(input_text)
            
            if self.tree_root is None:
                messagebox.showwarning("Warning", "Empty tree created!")
                return
            
            self.visualize_tree(show_centers=False)
            messagebox.showinfo("Success", "Tree built successfully!")
            
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {str(e)}")
    
    def solve_problem(self):
        """Solve the service center problem"""
        try:
            # Parse input
            input_text = self.input_entry.get()
            self.tree_root = TreeBuilder.parse_input(input_text)
            
            if self.tree_root is None:
                messagebox.showwarning("Warning", "Cannot solve for empty tree!")
                return
            
            # Solve
            solver = ServiceCenterSolver()
            details = solver.get_solution_details(self.tree_root)
            
            # Display results
            self.result_label.config(
                text=f"Minimum Service Centers: {details['total_centers']}"
            )
            
            # Show explanation
            self.explanation_text.delete(1.0, tk.END)
            self.explanation_text.insert(tk.END, details['explanation'])
            
            # Visualize tree with service centers highlighted
            self.visualize_tree(show_centers=True, center_values=details['center_locations'])
            
        except Exception as e:
            messagebox.showerror("Error", f"Error solving: {str(e)}")
    
    def visualize_tree(self, show_centers=False, center_values=None):
        """Draw the binary tree on canvas"""
        self.canvas.delete("all")
        self.canvas_nodes = {}
        
        if self.tree_root is None:
            self.canvas.create_text(
                225, 200,
                text="No tree to display",
                font=("Arial", 14),
                fill="white"
            )
            return
        
        if center_values is None:
            center_values = []
        
        # Calculate tree dimensions
        def get_tree_height(node):
            if not node:
                return 0
            return 1 + max(get_tree_height(node.left), get_tree_height(node.right))
        
        height = get_tree_height(self.tree_root)
        
        # Draw tree recursively
        def draw_node(node, x, y, dx, level):
            if node is None:
                return
      
            is_center = node.val in center_values if show_centers else False
            
           
            if node.left:
                child_x = x - dx
                child_y = y + 60
                self.canvas.create_line(x, y, child_x, child_y, fill="#00ff00", width=2)
                draw_node(node.left, child_x, child_y, dx // 2, level + 1)
            
            if node.right:
                child_x = x + dx
                child_y = y + 60
                self.canvas.create_line(x, y, child_x, child_y, fill="#00ff00", width=2)
                draw_node(node.right, child_x, child_y, dx // 2, level + 1)
            
 
            radius = 20
            color = "#ff4757" if is_center else "#00aaff"
            outline = "#ffff00" if is_center else "#00ff00"
            
            self.canvas.create_oval(
                x - radius, y - radius,
                x + radius, y + radius,
                fill=color,
                outline=outline,
                width=3
            )
     
            self.canvas.create_text(
                x, y,
                text=str(node.val),
                font=("Arial", 12, "bold"),
                fill="white"
            )
            
            if is_center:
                self.canvas.create_text(
                    x, y - 30,
                    text="🏢",
                    font=("Arial", 16)
                )
        
        canvas_width = 450
        start_x = canvas_width // 2
        start_y = 40
        initial_dx = canvas_width // 4
        
        draw_node(self.tree_root, start_x, start_y, initial_dx, 0)
    
    def clear_all(self):
        self.input_entry.delete(0, tk.END)
        self.input_entry.insert(0, "0, 0, null, 0, null, 0, null, null, 0")
        self.result_label.config(text="Minimum Service Centers: ---")
        self.explanation_text.delete(1.0, tk.END)
        self.canvas.delete("all")
        self.tree_root = None
        self.canvas_nodes = {}



if __name__ == "__main__":
    root = tk.Tk()
    app = ServiceCenterGUI(root)
    root.mainloop()